In [3]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd


# Import Dataset, Processing

In [67]:
DF = pd.read_csv('gurobiTime.csv',index_col=0)
DF = DF.drop(['player.1'],axis=1)


positionDF = []
num_positions = len(DF['position'].unique())


for i in DF['position'].unique():
    positionDF.append( (DF[DF['position'] == i]) )

    

[]

Make Gurobi datastructures to help represent the problem

We create dictionaries for every single variable for each player, to allow for efficient access and ease of use.  The keys are Players, the tupleList objects are returnPlayer, playerVariance, playerCost , and position


QB, RB, RB, WR, WR, WR, TE, D, K 



In [201]:
bigDict = {}
for df_iter in range(len(positionDF)):
    
    currPosition = positionDF[df_iter]  
    for player in currPosition.index:
        returnPlayer = currPosition.loc[player]['mean']
        playerVariance = currPosition.loc[player]['variance']
        playerCost = currPosition.loc[player]['cost']
        position = currPosition.loc[player]['position']
        bigDict[player] = [returnPlayer, playerVariance, playerCost , position]
         
player, returnPlayer, playerVariance, playerCost , position = gp.multidict(bigDict)


positionPlayerConstraintVars = gp.tuplelist( [(position.select(playr)[0],playr) for playr in player])

playerDecisionVars = gp.tuplelist( [ (choice, returnPlayer.select(choice)[0], playerVariance.select(choice)[0])  for choice in player ] )

positionConstraints = gp.tupledict( { 'QB':1, "RB":2,"WR":3, 'TE':1,"DEF":1,'K':1} )




In [239]:
positionPlayerConstraintVars



# Constraints: assign exactly shiftRequirements[s] workers to each shift s

# reqCts = {}
# for s in shifts:
#   reqCts[s] = m.addConstr(
#        gp.quicksum(x[w,s] for w,s in availability.select('*', s)) ==
#        shiftRequirements[s], s)
positionConstraints

{'QB': 1, 'RB': 2, 'WR': 3, 'TE': 1, 'DEF': 1, 'K': 1}

In [232]:
try:
    m = gp.Model('QMIP')
    #add binary decision variables on whether 
    
    
    #add decision variables
    players = m.addVars( playerDecisionVars, vtype=GRB.BINARY, name='players') 
    
    
    
    #add position constraint. We set upper bound to be one and keep the constraint a continuous variable since the 
    #model will keep integer solutions
    
    #positionConstratint = m.addVars(positionConstraints, ub =1 , name='positionConstr')
    
    
    # set objective funcition
    m.setObjective(  - gp.quicksum( returnPlayer[choice] *players[choice,retrn,var] for choice, retrn, var in players ) 
                   + gp.quicksum( playerVariance[choice] *players[choice,retrn,var] for choice, retrn, var in players ) 
                   ,GRB.MINIMIZE)
    
    #constraint: sum of each position must be a constraint:
    #for pos in DF['position'].unique():
        
        
    posConstrs = {} 
    for pos in DF['position'].unique():
        posConstrs[pos] = m.addConstr( 
        gp.quicksum(x[choice,retrn])
        )
        
        
    modelPositionConstr =  m.addConstrs(  players.sum(position[choice] ) ==  positionConstraints [ position[choice] ] for choice ,_,_ in players  ) 
    
    m.update()
    
    m.optimize()

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 478 rows, 478 columns and 0 nonzeros
Model fingerprint: 0x28161cb6
Variable types: 0 continuous, 478 integer (478 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [4e-14, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [200]:
x = position['Arizona'] 

positionConstraints

{'QB': 1, 'RB': 2, 'WR': 3, 'TE': 1, 'D': 1, 'K': 1}

2.7